<a href="https://colab.research.google.com/github/Aleksandr62aa/Computer_Vision/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Aleksandr62aa/Computer_Vision.git
%cd Computer_Vision

In [ ]:
# !pip install torch==2.2.1+cu121 torchvision==0.17.1+cu121  torchaudio==2.2.1+cu121
# !pip install torch

In [ ]:
!pip install -r requirements.txt

In [3]:
import yaml
import json

from nodes.VideoReader import VideoReader
from nodes.ShowNode import ShowNode
from nodes.VideoSaverNode import VideoSaverNode
# from nodes.DetectionTrackingNodes import DetectionTrackingNodes
from nodes.DetectionTrackingNodes_v1 import DetectionTrackingNodes
from nodes.TrackerInfoUpdateNode import TrackerInfoUpdateNode
from nodes.CalcStatisticsNode import CalcStatisticsNode
from nodes.SendInfoDBNode import SendInfoDBNode
from omegaconf import DictConfig, OmegaConf

def main() -> None:
    # # Чтение данных из configuration yaml (конфигурационный файл)
    with open('configs/app_config.yaml', 'r') as file:
        configuration = yaml.safe_load(file)
    config =json.loads(json.dumps(configuration, indent=2))

    video_reader = VideoReader(config["video_reader"])
    detection_node = DetectionTrackingNodes(config)
    tracker_info_update_node = TrackerInfoUpdateNode(config)
    calc_statistics_node = CalcStatisticsNode(config)
    show_node = ShowNode(config)
    video_saver_node = VideoSaverNode(config["video_saver_node"])

    save_video = config["pipeline"]["save_video"]
    send_info_db = config["pipeline"]["send_info_db"]


    if send_info_db:
        send_info_db_node = SendInfoDBNode(config)

    for frame_element in video_reader.process():

        frame_element = detection_node.process(frame_element)
        frame_element = tracker_info_update_node.process(frame_element)
        frame_element = calc_statistics_node.process(frame_element)

        if send_info_db:
            frame_element = send_info_db_node.process(frame_element)

        frame_element = show_node.process(frame_element)

        if save_video:
            video_saver_node.process(frame_element)

if __name__ == "__main__":
    main()

Детекция будет производиться на cuda
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients, 78.9 GFLOPs


Видео сохранено в папке video_out
